# Text Generation [Markov Chain]
---
- Author: Diego Inácio
- GitHub: [github.com/diegoinacio](https://github.com/diegoinacio)
- Notebook: [text-generation_markov-chain.ipynb](https://github.com/diegoinacio/creative-coding-notebooks/blob/master/ML-and-AI/text-generation_markov-chain.ipynb)
---
Text generation with [*Markov Chain*](https://en.wikipedia.org/wiki/Markov_chain), using characters as base.

A *Markov chain* is a sequence $x_1, x_2, x_3, ..., x_n$ of random variables - in our case, random characters. The distribution of the conditional probabilities of $x_{n+1}$ is a function of $x_n$, where:

$$ \large \displaystyle 
\Pr(X_{n+1}=x|X_{0},X_{1},X_{2},\ldots ,X_{n})=\Pr(X_{n+1}=x|X_{n})
$$

In [1]:
# Scraping libraries
import requests
from bs4 import BeautifulSoup

import numpy as np

## Data Scraping
---
As data, we are going to use lyrics from one of my favorite hard rock band.. **Van Halen**. To get those lyrics, let's scrape this data in the website [AllTheLyrics](https://www.allthelyrics.com) and process them to build a text corpus.

In [2]:
# Request and parse
PAGE = "https://www.allthelyrics.com"
URL = f'{PAGE}/lyrics/van_halen'
request = requests.get(URL).text
parse = BeautifulSoup(request, "html.parser")

In [3]:
%%time
np.random.seed(1)
# Define how many lyrics will be considered
# p = 0.1 indicates that 10% will be read
p = 0.2        # ! Parameter

# Init text corpus
TEXT = ""

# Find all 
ITEM = parse.find_all("li", {"class": "lyrics-list-item"})
ITEM_alpha = np.random.choice([0, 1], len(ITEM), p=[1 - p, p])

for i, item in enumerate(ITEM):
    link = item.find("a")
    if not link or not ITEM_alpha[i]:
        continue
    url = f'{PAGE}{link["href"]}'
    request_ = requests.get(url).text
    parse_ = BeautifulSoup(request_, "html.parser")
    TEXT += parse_.find("div", {"class": "content-text-inner"}).text
    print(f'{100*i/len(ITEM):>06.02f} %', end="\r")
TEXT = TEXT.replace("\n", " ")
print(f'{100:>06.02f} %')

100.00 %
Wall time: 23.4 s


In [4]:
# Find all characters and symbols in the text
SYMBOLS = sorted(set(TEXT))
SYMBOLS_N = len(SYMBOLS)
print(SYMBOLS)

[' ', '!', "'", '(', ')', ',', '-', '.', '0', '1', '3', '5', '9', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'W', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'º']


## Transition Matrix
---
[Transition](https://en.wikipedia.org/wiki/Markov_chain#Transitions) matrix contains all probilities for the next event, considering the current one. For example, considering the possibilities **A**, **B** and **C**:

|     |  A   |  B   |  C  |
| :-: | :--: | :--: | :-: |
|  A  | 0.1  | 0.6  | 0.3 |
|  B  | 0.25 | 0.05 | 0.7 |
|  C  | 0.7  | 0.3  |  0  |

If the current state is **A**, the probability of getting **B** is *0.6*.

In this case, the transition matrix has order 1. For nth-order matrices, we consider the last **n** states in sequence. For example, a 2nd-order matrix would be:

|     |  A   |  B   |  C  |
| :-: | :--: | :--: | :-: |
| AA  | 0.1  | 0.6  | 0.3 |
| AB  | 0.25 | 0.05 | 0.7 |
|  :  | ...  | ...  | ... |
| CA  | 0.7  | 0.3  |  0  |
|  :  | ...  | ...  | ... |

The higher the order, the more "readable" the generated text will be.

To build our transion matrix we have basically to calculate the probability of the states, depending on the order we want to use.

In [5]:
def order_sequences(TEXT, order):
    """
    Return all character sequences with length equal to the order
    """
    TEXT_N = len(TEXT)
    ORDERS = [TEXT[i:i + order] for i in range(TEXT_N - order)]
    return sorted(set(ORDERS))

In [6]:
def transition_matrix(TEXT, ORDERS, order):
    """
    Build transition matrix
    """
    ORDERS_N = len(ORDERS)
    P = np.ones([ORDERS_N, SYMBOLS_N])*1e-8
    for i, e1 in enumerate(TEXT[order:], order):
        i1 = SYMBOLS.index(e1)
        e2 = TEXT[i-order:i]
        i2 = ORDERS.index(e2)
        P[i2, i1] += 1
    P = P/P.sum(axis=1)[:,np.newaxis]
    return P

## Text Generation
---
Having the transition matrix, we can start building the text generation procedure. The first step is to define the inital state and ganerate the following states by using the probabilities and choosing randomly.

In [7]:
def text_generation(TEXT, order, lines):
    """
    Generate text
    """
    # Get transition matrix
    ORDERS = order_sequences(TEXT, order)
    P = transition_matrix(TEXT, ORDERS, order)
    # Count the occurences in the text, for each sequence
    # and define its probabilities
    ORDERS_count = np.array([TEXT.count(o) for o in ORDERS])
    ORDERS_p = ORDERS_count/ORDERS_count.sum()
    # Build text
    LINE = []
    for _ in range(lines):
        # Define initial chain of characters randomly
        line = np.random.choice(ORDERS, 1, p=ORDERS_p)[0]
        symbol = line[-1]
        line_s = np.random.randint(20, 50)
        while (len(line) < line_s) or (symbol not in [" ", ",", ".", "!"]):
            p = P[0]*0 + 1
            index0 = ORDERS.index(line[-order:])
            p *= P[index0]
            index1 = np.random.choice(SYMBOLS_N, 1, p=p/p.sum())[0]
            symbol = SYMBOLS[index1]
            line += symbol
        LINE += [line]
    return "\n".join(LINE)

## Examples
---
In order to visualize the impact of the order in the transition matrix, let's generate texts with multiple nth-order matrices.

### 1st-order Matrix
---
In this case, we just consider the current character probabilities, which is likely to produce a weird text.

In [8]:
np.random.seed(1)
print(text_generation(TEXT, 1, 8), end="\n\n")
print(text_generation(TEXT, 1, 8))

et y gelwn'scanseayour lous Heemag,
 t swilo) wal 10 be m me he, llivet St. wassthin'tayot 
veveroniee rere th is ike Gofa.
horer chth t t, sh! Pa. sh Ith, s myolinof I'shaye'stha 
d I f Totot sep paront (rad it's 
d k wn am t'vestsedomee mat wh, n het y'verehangouronghanallive 
e Bure choh Do Ifisol co oheterodan're 
 buin' lieede s lo drorowno) rer 

p d oune ived,iguppathet 
our iny bere k t Anongheve reve I Nouronooun I 
 maywatyel yowh sot ckndicitheromyelitroun'veas 
unt st lerende (d lsainttow,
 f t y h, in' St mu if sis 
 fu youngh fend, (ray 
bakn ) t'mowevere t 
buplilirdr s sat'.. st ag g Oou betin'tawhe 


### 3rd-order Matrix
---
In this case, we consider the last 3 characters probabilities, which is likely to produce a more comprehensive words.

In [9]:
np.random.seed(1)
print(text_generation(TEXT, 3, 8), end="\n\n")
print(text_generation(TEXT, 3, 8))

e... what you know mine, lovin' 
led, round, round, near all 
 stillin', I'd, round, round, 
ound, For divide it ride your belin' Go some down 
ring in me world Give You Oucheeliever Uh,
, Are mone body sharely Only to how who 
 ooo) what you wake it wer be the 
 be subtle with someone can puts 

ed a wheek a 3 piece ream,
 give else what want life You'll me everywhen the 
pail Sweet it hey you want 
ook in ove and neve it nothing you know show magic,
 must turn you Dig deny 
on! Dream, get she's comind want little We're 
ed upon the comind the a stop lost of want faithough 
m feeliever a litter (Ooooh oooh 


### 7th-order Matrix
---
In this case, we consider the last 7 characters probabilities, which is likely to produce a little more comprehensive sentences.

In [10]:
np.random.seed(1)
print(text_generation(TEXT, 7, 8), end="\n\n")
print(text_generation(TEXT, 7, 8))

e. Standing on top of the women 
see baby now! Hey, you don't always been 
o live Give to live Give 
n'....no, don't know, Cause it's gonna stop 
 Love hurts you sometimes 
ed up in lights Everything you've got to give You've 
st once born, can't ever lose faith 
 I wonder who I'd rather be if I had one wish 

o try, oh could it ever 
ce can't refuse I wouldn't know and 
st believe a little If you want faith, you just 
or is a little while your head,
round) Run-run-runaround, round) 
ever had no room to second guess puts me under 
thing's gonna like it Hey! Alright! Whoo! How 'bout 
ed Oh, what a fool believe You can always make 
